<a href="https://colab.research.google.com/github/RO-AD/waymo-od-motion-pred/blob/main/tutorial/4_dataset-structure/hj-dataset-structure.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

tutorial/4_dataset-structure/hj-dataset-structure.ipynb

# Dataset Proto 구조 분석

## 환경세팅

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
VALIDATION_PATH = "/content/drive/MyDrive/waymo-od-dataset/motion_v_1_2_0/uncompressed/scenario/validation/"

In [3]:
%%capture
!pip3 install --upgrade pip
!pip install waymo-open-dataset-tf-2-11-0==1.5.1 # 최신 버전 라이브러리

In [4]:
import math
import os
import uuid
import time

import numpy as np
import tensorflow as tf

from google.protobuf import text_format
from waymo_open_dataset.metrics.ops import py_metrics_ops
from waymo_open_dataset.metrics.python import config_util_py as config_util
from waymo_open_dataset.protos import motion_metrics_pb2

In [7]:
filename = VALIDATION_PATH + 'validation.tfrecord-00000-of-00150'

In [8]:
from waymo_open_dataset.protos import scenario_pb2 

dataset = tf.data.TFRecordDataset(filename, compression_type='')
for data in dataset :
   scenario = scenario_pb2.Scenario()
   scenario.ParseFromString(data.numpy())
   break

In [9]:
scenario.scenario_id

'19a486cd29abd7a7'

## Dataset 구조 분석
waymo 공식 깃허브에 있는 proto 파일을 보며 Dataset 구조 분석 해보기
- [scenario.proto](https://github.com/waymo-research/waymo-open-dataset/blob/master/src/waymo_open_dataset/protos/scenario.proto)
- [map.proto](https://github.com/waymo-research/waymo-open-dataset/blob/master/src/waymo_open_dataset/protos/map.proto)
- [compressed_lidar.proto](https://github.com/waymo-research/waymo-open-dataset/blob/master/src/waymo_open_dataset/protos/compressed_lidar.proto)

### scenario.proto

```proto
// This proto contains the  Waymo Open Dataset Motion source data.

syntax = "proto2";

package waymo.open_dataset;

import "waymo_open_dataset/protos/compressed_lidar.proto";
import "waymo_open_dataset/protos/map.proto";
```

`compressed_lidar.proto`와 `map.proto`를 import 한다.

```proto
message Scenario {
  reserved 9;

  optional string scenario_id = 5;
  repeated double timestamps_seconds = 1;
  optional int32 current_time_index = 10;
  repeated Track tracks = 2;
  repeated DynamicMapState dynamic_map_states = 7;
  repeated MapFeature map_features = 8;
  optional int32 sdc_track_index = 6;
  repeated int32 objects_of_interest = 4;
  repeated RequiredPrediction tracks_to_predict = 11;
  repeated CompressedFrameLaserData compressed_frame_laser_data = 12;
}
```

하나씩 값을 보며 분석
```proto
// scenario_id
// The unique ID for this scenario. (옵션)
optional string scenario_id = 5;
```


In [23]:
scenario.scenario_id

'19a486cd29abd7a7'

```proto
// timestamps_seconds
// 0.0부터 0.1초 간격으로 9초까지 91개의 값을 가지고 있는 리스트
repeated double timestamps_seconds = 1;
```

In [24]:
scenario.timestamps_seconds

[0.0, 0.10004, 0.20005, 0.3, 0.40001, 0.49998, 0.59998, 0.69998, 0.79999, 0.89999, 0.99999, 1.09995, 1.19996, 1.29992, 1.39991, 1.49988, 1.59987, 1.69992, 1.79991, 1.89994, 1.99994, 2.09994, 2.19994, 2.29995, 2.39998, 2.49998, 2.59998, 2.69998, 2.79998, 2.89998, 2.99998, 3.10001, 3.20005, 3.30009, 3.40013, 3.50013, 3.60013, 3.70017, 3.80017, 3.90017, 4.00017, 4.10018, 4.20014, 4.30011, 4.40006, 4.50002, 4.59999, 4.69995, 4.79992, 4.89988, 4.99988, 5.09984, 5.19984, 5.29984, 5.39984, 5.49988, 5.59989, 5.69993, 5.79993, 5.89996, 6.0, 6.1, 6.2, 6.30003, 6.40003, 6.5, 6.6, 6.7, 6.8, 6.9, 6.99997, 7.09997, 7.19996, 7.29993, 7.39988, 7.49988, 7.59988, 7.69988, 7.79988, 7.89992, 7.99992, 8.09992, 8.19993, 8.29996, 8.39997, 8.49997, 8.59997, 8.69997, 8.79998, 8.89998, 8.99994]

```proto
// current_time_index
// 현재 시간의 인덱스. timestamps_seconds[10]은 약 1.0이다 (옵션)
optional int32 current_time_index = 10;
```


In [25]:
scenario.current_time_index

10

```proto
// tracks
// 시나리오의 모든 개체 트랙 리스트로, len(tracks[i].states)은 91로 동일하다. 
// tracks[i].states[j]는 timestamps_seconds[j] 시간에 해당하는 i번째 에이전트의 상태이다.
repeated Track tracks = 2;
```

In [30]:
print(len(scenario.tracks))
print(len(scenario.tracks[0].states))
print(scenario.tracks[0].id)
print(scenario.tracks[0].states[2])

11
91
368
center_x: 8382.134765625
center_y: 7209.99072265625
center_z: -13.698562246539266
length: 4.469150066375732
width: 1.9771449565887451
height: 1.4942156076431274
heading: -1.5596644878387451
velocity_x: 0.361328125
velocity_y: -19.1748046875
valid: true



```proto
// dynamic_map_states
// 시나리오의 동적 맵 상태 리스트 (예: 교통 신호 상태)
// 이 필드의 길이는 timestamp_seconds와 동일하다
// dynamic_map_states[i]는 timestamps_seconds[i] 시 동적 맵 상태를 인덱싱한다
repeated DynamicMapState dynamic_map_states = 7;
```

In [36]:
scenario.dynamic_map_states[0]

lane_states {
  lane: 120
  state: LANE_STATE_GO
  stop_point {
    x: 8391.668352965295
    y: 7316.195065592189
    z: -14.848865196389456
  }
}
lane_states {
  lane: 121
  state: LANE_STATE_GO
  stop_point {
    x: 8394.981797191336
    y: 7316.2616120958155
    z: -14.868865196389438
  }
}
lane_states {
  lane: 122
  state: LANE_STATE_GO
  stop_point {
    x: 8394.981797191336
    y: 7316.2616120958155
    z: -14.868865196389438
  }
}
lane_states {
  lane: 123
  state: LANE_STATE_GO
  stop_point {
    x: 8397.46688036219
    y: 7316.206156676259
    z: -14.92886519638944
  }
}

```proto
// map_features
// 시나리오의 정적 맵 기능 리스트 집합
repeated MapFeature map_features = 8;
```

In [39]:
scenario.map_features[0].id

3

In [42]:
scenario.map_features[0].road_edge.polyline[0]

x: 8292.619348114988
y: 7179.778677513115
z: -14.258546045464358

```proto
// sdc_track_index
// sdc : Self-Driving Car
// 맵 정보를 촬영하는 자율주행 차량 개체의 track 리스트에 해당하는 인덱스 (옵션)
// 보행자, 자전거, 자동차 등의 에이전트를 포함한 track 리스트 중 해당 인덱스는 무시
optional int32 sdc_track_index = 6;
```

In [43]:
scenario.sdc_track_index

10

```proto
// objects_of_interest
// 상호작용(interactive) 동작이 감지된 scene의 객체(Object) ID 목록 리스트
// 이 필드에 포함된 ID는 tracks 필드의 ID에 해당되며, tracks 필드의 객체와 일치한다
// 예를 들어, 자율주행 자동차가 도로를 따라 이동하는 동안, 
// 이 필드는 차량 주위에 위치한 보행자나 자전거 등과 같은 상호작용 객체 ID를 포함할 수 있다
// 이러한 객체들은 자율주행 자동차와의 상호작용을 감지하는 데 사용될 수 있다.
repeated int32 objects_of_interest = 4;
```

In [46]:
scenario.objects_of_interest

[]

```proto
// tracks_to_predict
// 예측을 생성하려는 추적 대상 객체 ID의 목록 리스트
// 해당 필드는 추적 대상 객체를 학습하는 데 사용된다
// tracks[tracks_to_predict.track_index].states[15::5]의 center_x, center_y를 예측해야 한다
// 즉, future timestamp인 1.5초부터 0.5초 간격으로 states의 center_x, center_y를 예측해야 한다 
repeated RequiredPrediction tracks_to_predict = 11;
```

In [47]:
scenario.tracks_to_predict

[track_index: 2
difficulty: LEVEL_1
]

```proto
// compressed_frame_laser_data
// time step에서의 Lidar 데이터를 담고 있다
// compressed_frame_laser_data[i]는 timestamps_seconds[i]에서의 상태를 나타낸다
// 이 필드는 현재 시간 인덱스보다 작거나 같은 모든 시간 단계에 대한 데이터를 포함한다
repeated CompressedFrameLaserData compressed_frame_laser_data = 12;
```

In [48]:
scenario.compressed_frame_laser_data

[]